In [ ]:
# has to currently run on cpu because I left this running for too long in colab

import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 12.3 MB/s eta 0:00:00


In [ ]:
# uploading the dataset from my local computer

from google.colab import files
uploaded = files.upload()

Saving news headlines sarcasm dataset.zip to news headlines sarcasm dataset.zip


Unzip the dataset to the file system. You can browse the file system of the Colab instance in the sidebar on the left.

In [ ]:
# unzipping the file that was just uploaded

!unzip "news headlines sarcasm dataset.zip" -d "news headlines sarcasm dataset"

Archive:  news headlines sarcasm dataset.zip
  inflating: news headlines sarcasm dataset/Sarcasm_Headlines_Dataset.json  
  inflating: news headlines sarcasm dataset/Sarcasm_Headlines_Dataset_v2.json  


In [ ]:
# reading in the json file to a pandas dataframe

import pandas as pd

df = pd.read_json("news headlines sarcasm dataset/Sarcasm_Headlines_Dataset_v2.json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
# finding the total number of samples in the datset
print('Number of headlines: {:,}\n'.format(df.shape[0]))

# displays 10 samples from the dataset
df.sample(10)

Number of headlines: 28,619



,is_sarcastic,headline,article_link
1420,0,"cruz hits back at 'cronyist, washington cartel...",https://www.huffingtonpost.comhttp://www.polit...
22418,1,date rapist tossing his mortarboard into air 3...,https://www.theonion.com/date-rapist-tossing-h...
15759,0,9 page-turners too addictive to put down,https://www.huffingtonpost.com/entry/9-page-tu...
2405,0,having kids radically reshapes parents' immune...,https://www.huffingtonpost.com/entry/having-ki...
852,0,powerball ticket sold with all winning numbers...,https://www.huffingtonpost.com/entry/powerball...
9032,1,hillary clinton spends busy day fueling specul...,https://politics.theonion.com/hillary-clinton-...
28322,1,comeback much harsher than insult,https://www.theonion.com/comeback-much-harsher...
12683,1,clairvoyant vince vaughn accepts movie role be...,https://entertainment.theonion.com/clairvoyant...
22322,0,the science-backed ways that movement boosts ...,https://www.huffingtonpost.com/entry/movement-...
15177,0,terrifying tornado gives couple a proposal sto...,https://www.huffingtonpost.com/entry/storm-cha...


In [ ]:
# drops the article link column because it is unneeded to read the dataset
df.loc[df.is_sarcastic == 0].sample(5)[['is_sarcastic', 'headline']]

,is_sarcastic,headline
20439,0,ecstasy and despair on this historic day
11800,0,'out of sight': 360-degree film series on dise...
2988,0,"huffpost rise morning newsbrief, october 20"
23051,0,how do i make my 4-month-old fall in love with...
27377,0,nra's ominous but misleading appeal


In [ ]:
# returns the numpy representation of headlines and labels in the dataset
# both will be of length 28619

headlines = df.headline.values
labels = df.is_sarcastic.values

In [ ]:
# determining the longest headline in the dataset so we can allocate a good number for padding

word_len=[]
for i in range(len(headlines)):
    word_len.append(len(headlines[i].split(' ')))
print("Max word length of the headlines:",max(word_len))

Max word length of the headlines: 151


In [ ]:
# converts bert through a Keras layer

import tensorflow_hub as hub 
!pip install bert-tensorflow

module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# adds the tokenizers (DistilBertTokenizer is slightly faster for these purposes)

import transformers

from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer


Tokenizer(vocabulary_size=30522, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [ ]:
import numpy as np
import tensorflow as tf

# encodes all headlines into the input sequence with the padding characters [CLS] and [SEP]

headlines = df.headline.values
labels = df.is_sarcastic.values

def bert_encode(headlines, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for headline in headlines:
        # convert headline into tokenized ids
        headline = tokenizer.tokenize(headline)
            
        # padding each headline with the special characters and then adding 0s to the rest of the array to fill max length
        headline = headline[:max_len-2]
        input_sequence = ["[CLS]"] + headline + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        # creates each token in the headline
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        # creates a mask to compare in length to the token headlines
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        # the full array with just zeroes
        segment_ids = [0] * max_len
        
        # add each array to 
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)



def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    # runs through each layer in the neural network from an input of 164 down to 1
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    # relu is the activation function for each layer
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(1, activation='relu')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    # learning rate as specificied as advised by BERT
    model.compile(tf.keras.optimizers.Adam(lr=5e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

max_len = 164
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(df.headline, df['is_sarcastic'], 
                                                                          test_size=0.1, random_state=0)
X_train = bert_encode(X_train, tokenizer, max_len=max_len)
X_val = bert_encode(X_val, tokenizer, max_len=max_len)

In [ ]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 164)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 164)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 164)]        0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     multiple             109482241   ['input_word_ids[0][0]',         
                                                                  'input_mask[0][0]',       

In [ ]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train, 
    validation_split=0.1,
    epochs=2,
    callbacks=[checkpoint, earlystopping],
    batch_size=16,
    verbose=1
)

Epoch 1/2
1449/1449 [==============================] - ETA: 0s - loss: 1.0029 - accuracy: 0.5776
Epoch 1: val_accuracy improved from -inf to 0.83657, saving model to model.h5
1449/1449 [==============================] - 819s 562ms/step - loss: 1.0029 - accuracy: 0.5776 - val_loss: 0.5789 - val_accuracy: 0.8366
Epoch 2/2
1449/1449 [==============================] - ETA: 0s - loss: 0.8085 - accuracy: 0.5265
Epoch 2: val_accuracy did not improve from 0.83657
1449/1449 [==============================] - 807s 557ms/step - loss: 0.8085 - accuracy: 0.5265 - val_loss: 0.7256 - val_accuracy: 0.5008
CPU times: user 17min 4s, sys: 3min 8s, total: 20min 12s
Wall time: 27min 5s


In [ ]:
%%time
model.load_weights('model.h5')
y_preds = model.predict(X_val).round().astype(int)
print("Validation accuracy: ", sklearn.metrics.accuracy_score(y_val, y_preds))

90/90 [==============================] - 33s 367ms/step
Validation accuracy:  0.843466107617051
CPU times: user 33.1 s, sys: 223 ms, total: 33.3 s
Wall time: 41.9 s
